# XtEHR Logical model processing

* ophalen XtEHR Logical Models
* consistente layout met zibs
* klaar voor een mapping - zie mapping Notebook

## Relevante kern content

NL kernset:
- Behandelaanwijzing

ePS required:
- Medicatie
- Diagnose/ Probleemlijst(actief)
- Verrichtingen(<6mnd, ≥ 6mnd)
- Allergieën en intoleranties + - - 
- Alerts
- Hulpmiddel
- Patiënt

ePS recommended + optional:
- Vaccinations
- Implants
- Treatment restrictions
- Vital signs
- History of illness
- Pregnancy
- Social history
- Functional status
- Results
- Plan of Care
- Encounters

# Imports

In [1]:
import pandas as pd
import requests
import json
import os
from io import StringIO

# Relevante modellen

Print een lijstje met de relevante Xt-EHR-modellen.

In [17]:
xtehrlist = ['Address', 'Alert', 'AllergyIntolerance', 'BodyStructure', 'Condition', 'DataSet', 'Device', 'DeviceUse', 'HealthProfessional', 'HumanName', 'Location', 'Medication', 'Organisation', 'Patient', 'Procedure', 'Telecom']
for i in range(len(xtehrlist)):
    print(i, xtehrlist[i])


0 Address
1 Alert
2 AllergyIntolerance
3 BodyStructure
4 Condition
5 DataSet
6 Device
7 DeviceUse
8 HealthProfessional
9 HumanName
10 Location
11 Medication
12 Organisation
13 Patient
14 Procedure
15 Telecom


# Kies het relevante model

* Vul het nummer bij het model hierboven in als `model_num`


In [18]:
model_num = 5
model = xtehrlist[model_num]
print(model)

file_path = os.path.join('xtehr', 'downloads', model + '.json')

DataSet


# Download het model

* Alleen nodig als dit nog niet gedaan is of als de analyse opnieuw gedaan moet worden.

In [21]:
xtehrbase = 'https://build.fhir.org/ig/Xt-EHR/xt-ehr-common/StructureDefinition-EHDS'
url = xtehrbase + model + '.json'
response = requests.get(url)
jsondata = response.json()

if (not os.path.exists('xtehr')):
    os.makedirs('xtehr')
with open(file_path, 'w') as jsonfile:
    json.dump(jsondata, jsonfile, indent=4)

# Kies en lees het te verwerken Logical Model

* Toon en verifieer dat de uitvoer er is

In [22]:
with open(file_path) as jsonfile:
    xlm = json.load(jsonfile)['snapshot']['element']
dfx = pd.DataFrame(xlm)
dfx

,id,path,short,definition,min,max,base,isModifier,mapping,type,binding
0,EHDSDataSet,EHDSDataSet,DataSet model,Common elements (including header) for all doc...,0,*,"{'path': 'Base', 'min': 0, 'max': '*'}",False,"[{'identity': 'rim', 'map': 'n/a'}]",NaN,NaN
1,EHDSDataSet.header,EHDSDataSet.header,Common header for all patient-related data,Common header for all patient-related data,1,1,"{'path': 'EHDSDataSet.header', 'min': 1, 'max'...",NaN,NaN,[{'code': 'Base'}],NaN
2,EHDSDataSet.header.subject,EHDSDataSet.header.subject,Subject,Patient/subject information,1,1,"{'path': 'EHDSDataSet.header.subject', 'min': ...",NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
3,EHDSDataSet.header.identifier,EHDSDataSet.header.identifier,Business identifier for the object,Business identifier for the object,0,*,"{'path': 'EHDSDataSet.header.identifier', 'min...",NaN,NaN,[{'code': 'Identifier'}],NaN
4,EHDSDataSet.header.authorship,EHDSDataSet.header.authorship,Authorship,Resource authoring details,1,*,"{'path': 'EHDSDataSet.header.authorship', 'min...",NaN,NaN,[{'code': 'Base'}],NaN
5,EHDSDataSet.header.authorship.author[x],EHDSDataSet.header.authorship.author[x],Author,Author(s) by whom the resource was/were author...,1,1,{'path': 'EHDSDataSet.header.authorship.author...,NaN,NaN,[{'code': 'https://www.xt-ehr.eu/specification...,NaN
6,EHDSDataSet.header.authorship.datetime,EHDSDataSet.header.authorship.datetime,Date and time of authoring/issuing,Date and time of the issuing the document/reso...,1,1,{'path': 'EHDSDataSet.header.authorship.dateti...,NaN,NaN,[{'code': 'dateTime'}],NaN
7,EHDSDataSet.header.lastUpdate,EHDSDataSet.header.lastUpdate,Date and time of the last update to the resource,Date and time of the last update to the docume...,0,1,"{'path': 'EHDSDataSet.header.lastUpdate', 'min...",NaN,NaN,[{'code': 'dateTime'}],NaN
8,EHDSDataSet.header.status,EHDSDataSet.header.status,Status of the resource,Status of the resource,1,1,"{'path': 'EHDSDataSet.header.status', 'min': 1...",NaN,NaN,[{'code': 'CodeableConcept'}],NaN
9,EHDSDataSet.header.statusReason[x],EHDSDataSet.header.statusReason[x],Reason for the current status of the resource.,Reason for the current status of the resource.,0,1,"{'path': 'EHDSDataSet.header.statusReason[x]',...",NaN,NaN,"[{'code': 'CodeableConcept'}, {'code': 'string'}]",NaN


# Opschonen uitvoer zodat zib en xtehr consistent zijn

* een card. veld met n..m
* type (is een JSON object) omzetten naar tekst
* alleen nodige kolommen houden

In [23]:
dfx.insert(2, 'card.', None)
dfx['card.'] = dfx['min'].astype(str) + '..' + dfx['max'].astype(str)
for i, row in dfx.iterrows():
    if 'type' in row:
        # row['type'][0]['code']
        try:
            dfx.at[i, 'type'] = row['type'][0]['code'].split('/')[-1]
        except TypeError:
            pass
dfx= dfx[['id', 'path', 'short', 'definition', 'type', 'card.', 'binding']]
dfx

,id,path,short,definition,type,card.,binding
0,EHDSDataSet,EHDSDataSet,DataSet model,Common elements (including header) for all doc...,NaN,0..*,NaN
1,EHDSDataSet.header,EHDSDataSet.header,Common header for all patient-related data,Common header for all patient-related data,Base,1..1,NaN
2,EHDSDataSet.header.subject,EHDSDataSet.header.subject,Subject,Patient/subject information,EHDSPatient,1..1,NaN
3,EHDSDataSet.header.identifier,EHDSDataSet.header.identifier,Business identifier for the object,Business identifier for the object,Identifier,0..*,NaN
4,EHDSDataSet.header.authorship,EHDSDataSet.header.authorship,Authorship,Resource authoring details,Base,1..*,NaN
5,EHDSDataSet.header.authorship.author[x],EHDSDataSet.header.authorship.author[x],Author,Author(s) by whom the resource was/were author...,EHDSHealthProfessional,1..1,NaN
6,EHDSDataSet.header.authorship.datetime,EHDSDataSet.header.authorship.datetime,Date and time of authoring/issuing,Date and time of the issuing the document/reso...,dateTime,1..1,NaN
7,EHDSDataSet.header.lastUpdate,EHDSDataSet.header.lastUpdate,Date and time of the last update to the resource,Date and time of the last update to the docume...,dateTime,0..1,NaN
8,EHDSDataSet.header.status,EHDSDataSet.header.status,Status of the resource,Status of the resource,CodeableConcept,1..1,NaN
9,EHDSDataSet.header.statusReason[x],EHDSDataSet.header.statusReason[x],Reason for the current status of the resource.,Reason for the current status of the resource.,CodeableConcept,0..1,NaN


# Opslaan in Excel

In [26]:
url = os.path.join('xtehr', 'mappable', model + '.xlsx')
dfx.to_excel(url, index=False)